In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, Input, Flatten, concatenate,LSTM,Dropout
from tensorflow.keras.models import Model
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/trip_advisor_data.csv')
data.head()

,overall,cleanliness,location,rooms,service,value,title,text,username,num_cities,num_helpful_votes,num_reviews,num_type_reviews,id,date_stayed,offering_id,date,via_mobile,user_id
0,4.0,5.0,5.0,5.0,4.0,3.0,“Its the best of the Andaz Brand in the US....”,I have stayed at each of the US Andaz properti...,JamesE339,34.0,0,104.0,49.0,147612823,December 2012,1762573,"December 17, 2012",False,BA524A238B1171206691A6CC3F28F266
1,4.0,5.0,5.0,5.0,4.0,5.0,“Stunningly Wonderful!”,Other hotels in NYC that are space challenged ...,romadaro,70.0,5,103.0,85.0,147531970,December 2012,1762573,"December 16, 2012",False,C81AB7D49D98FA410EA191E15F427BEC
2,5.0,5.0,4.0,5.0,5.0,3.0,“Unexpectedly good”,"Campus hotels are a little unpredictable, but ...",Grotshops,35.0,0,96.0,51.0,140624089,September 2012,108038,"September 18, 2012",False,4C8EFD12CA2F03E9D85709814BB16501
3,4.0,4.0,3.0,4.0,4.0,4.0,"“Pleasant enough, but a bit overrated”",For our most recent ever-so-brief New York vis...,pillowsofwanderlust,20.0,2,57.0,49.0,145940048,November 2012,1776857,"November 22, 2012",False,3F88F8D4F220EFE6418ED0B22AAF03E6
4,4.0,4.0,4.0,5.0,4.0,4.0,“nice suprise”,Booked this hotel thinking it was another Hilt...,beatricepugzilla,28.0,2,41.0,36.0,127060765,March 2012,108038,"April 1, 2012",False,A90E45A9CAD3E818327172C16C27BFB1


In [ ]:
data = data[data.location.isin(['0.0','1.0','2.0','3.0','4.0','5.0'])]

In [ ]:
cls = ['cleanliness','location','rooms','service','value']
for cl in cls:
  data[cl] = np.asarray(data[cl]).astype(np.float64)

for cl in cls:
  print(cls,data[data[cl]==1.0].shape,data[data[cl]==2.0].shape,data[data[cl]==3.0].shape,data[data[cl]==4.0].shape,data[data[cl]==5.0].shape)

['cleanliness', 'location', 'rooms', 'service', 'value'] (307, 19) (349, 19) (991, 19) (2497, 19) (5524, 19)
['cleanliness', 'location', 'rooms', 'service', 'value'] (137, 19) (251, 19) (934, 19) (2360, 19) (5986, 19)
['cleanliness', 'location', 'rooms', 'service', 'value'] (413, 19) (579, 19) (1556, 19) (2978, 19) (4142, 19)
['cleanliness', 'location', 'rooms', 'service', 'value'] (486, 19) (446, 19) (1160, 19) (2341, 19) (5235, 19)
['cleanliness', 'location', 'rooms', 'service', 'value'] (522, 19) (610, 19) (1586, 19) (3009, 19) (3941, 19)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
ord_enc = OrdinalEncoder()
data['offering_id'] =ord_enc.fit_transform(data['offering_id'].values.reshape(-1, 1)).reshape(1,-1)[0]
data['user_id'] =ord_enc.fit_transform(data['user_id'].values.reshape(-1, 1)).reshape(1,-1)[0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
import re
def remove_punc(line):
  line = re.sub('[0-9@]', '',line)
  line = re.sub('[#[]', '',line)
  line = re.sub(']', '',line)
  line = re.sub('[\'":,.?!&)(]', '',line)
  line = line.rstrip().lstrip()
  return line

data['text'] = data['text'].apply(lambda x : remove_punc(x))
data.text.values

array(['I have stayed at each of the US Andaz properties and this one is the best of them all MUCH better than the West Hollywood property where Brand standards are slowly fading This is the shining star here in the States\nMy room suite on the th floor was fantastic as usual Perfectly maintained very comfortable great bathrooms best in NYC The bathroom is huge with large shower room and multiple shower heads Double lavatories flank one wall Huge\nTHe bed is always perfect Televisions in both rooms were perfect for my limited time spend in the suite Nice desk for work area and in room dining \nEvery room has a fridge with complimentary soft drinks snacks water sparkling water milk and juices Nice touch Suite has full size refrigerator with icemaker and also features a microwave sink and dishwasher drawerall nicely concealed behind beautiful cabinetry Really great for a longer stay\nI do have  complaints First the elevators The waits were sometimes ridiculous  elevators serve the main t

In [ ]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

In [ ]:
import nltk
from nltk.corpus import stopwords
stop_w = stopwords.words('english')

from nltk.stem import WordNetLemmatizer
  
lemmatizer = WordNetLemmatizer()

data['text'] = data['text'].apply(lambda x : ' '.join([lemmatizer.lemmatize(w.lower()) for w in x.split(' ') if lemmatizer.lemmatize(w.lower()) not in stop_w ]))
data['text'] = data['text'].apply(lambda x : ' '.join([t for t in x.split(' ') if len(t)>0]))
    
data.text.values

array(['stayed u andaz property one best much better west hollywood property brand standard slowly fading shining star states\nmy room suite th floor wa fantastic usual perfectly maintained comfortable great bathroom best nyc bathroom huge large shower room multiple shower head double lavatory flank one wall huge\nthe bed always perfect television room perfect limited time spend suite nice desk work area room dining \nevery room ha fridge complimentary soft drink snack water sparkling water milk juice nice touch suite ha full size refrigerator icemaker also feature microwave sink dishwasher drawerall nicely concealed behind beautiful cabinetry really great longer stay\ni complaint first elevator wait sometimes ridiculous elevator serve main tower rather limited number room newer hotel service elevator wa continually surprised waiting elevator many stop made housekeeping enter cart room service cart tray etc could hyatt skip service elevator new property really stink timesits bad\nlastl

In [ ]:
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.layers import Dense, LSTM, Embedding,Bidirectional
import tensorflow
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
def mod():
  sent_inp = Input(shape=(150, 768))
  X = LSTM(512, return_sequences=True)(sent_inp)
  
  X = LSTM(100,return_sequences=True)(X)
  X = Dropout(0.2)(X)
  X = LSTM(200,return_sequences=True)(X)
  X = Dropout(0.2)(X)
  X = LSTM(100)(X)
  
  X = Dense(16,activation='relu',kernel_regularizer='l1')(X) 
  out = Dense(1,activation='sigmoid')(X)

 
  


  
  
  
  
  model = Model(inputs=sent_inp, 
                  outputs=out)
  
  return model


In [ ]:
model_sent = mod()

In [ ]:
from tensorflow.keras.models import load_model
model_sent = load_model('/content/drive/MyDrive/Trip_model_sentimentv2.h5')

In [ ]:
!pip install --upgrade transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 26.2 MB/s 
     |████████████████████████████████| 101 kB 14.5 MB/s 
     |████████████████████████████████| 6.6 MB 46.7 MB/s 
     |████████████████████████████████| 596 kB 65.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import BertTokenizer, TFBertModel,pipeline
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained("bert-base-uncased")


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
texts = data.text.values
i=0
pred = []
for text in texts :
  i+=1
  if i%1000==0:
    print(len(texts)-i)
  encoded_input = tokenizer([text], return_tensors='tf',max_length = 150,  # maximum length of a sentence  (TODO Figure the longest passage length)
      truncation = 'longest_first',padding='max_length'
    )
    
  output = model(encoded_input)
  prd = model_sent.predict(output['last_hidden_state'])[0]
  pred.append(1 if prd[0]>0.5 else 0)
  

8668
7668
6668
5668
4668
3668
2668
1668
668


In [ ]:
pred

[1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,


In [ ]:
data['sentiment_score'] = pred

In [ ]:
data.to_csv('trip_data.csv')

#split data into train test for deployment

In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data,test_size=0.2,random_state=123)
train_data.to_csv('data_simple_sent.csv',index=False)
test_data[['user_id','offering_id']].to_csv('test_data.csv',index=False)

#kfold

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5,random_state=24,shuffle=True)

In [ ]:
f=0
for train_index, test_index in kf.split(data):
  print('fold number : ',f)
  f+=1
  train_data , test_data = data.iloc[train_index,:] , data.iloc[test_index,:]
  train_data.to_csv('/content/drive/MyDrive/Trip_fold_with_sentiment'+str(f)+'_train.csv',index=False)
  test_data.to_csv('/content/drive/MyDrive/Trip_fold_with_sentiment'+str(f) +'_test.csv',index=False)

fold number :  0
fold number :  1
fold number :  2
fold number :  3
fold number :  4


In [ ]:
train_data

,overall,cleanliness,location,rooms,service,value,title,text,username,num_cities,num_helpful_votes,num_reviews,num_type_reviews,id,date_stayed,offering_id,date,via_mobile,user_id,sentiment_score
1,4.0,5.0,5.0,5.0,4.0,5.0,“Stunningly Wonderful!”,hotel nyc space challenged would well take not...,romadaro,70.0,5,103.0,85.0,147531970,December 2012,1927.0,"December 16, 2012",False,677.0,1
2,5.0,5.0,4.0,5.0,5.0,3.0,“Unexpectedly good”,campus hotel little unpredictable one star roo...,Grotshops,35.0,0,96.0,51.0,140624089,September 2012,849.0,"September 18, 2012",False,271.0,0
3,4.0,4.0,3.0,4.0,4.0,4.0,"“Pleasant enough, but a bit overrated”",recent ever-so-brief new york visit day night ...,pillowsofwanderlust,20.0,2,57.0,49.0,145940048,November 2012,1929.0,"November 22, 2012",False,224.0,1
6,4.0,4.0,4.0,3.0,5.0,4.0,“Service was outstanding...dont let decor get ...,like everyone ha posted physical structure dat...,bobolito,28.0,0,66.0,52.0,143708004,October 2012,682.0,"October 25, 2012",False,818.0,1
8,5.0,5.0,5.0,5.0,5.0,5.0,“Outstanding Service and facility! If you want...,outstanding great staff service facility conve...,Tmayfield1964,NaN,1,1.0,NaN,140149098,September 2012,682.0,"September 13, 2012",False,412.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10044,3.0,4.0,4.0,4.0,2.0,5.0,“Great Location. Terrible Service”,friend booked hotel weekend evening night birt...,reeseoreo,NaN,1,1.0,NaN,25853064,March 2009,353.0,"March 9, 2009",False,173.0,0
10045,5.0,5.0,5.0,4.0,5.0,3.0,"“Great Location, would stay again”",nice hotel decor might everyones taste wa quit...,CityGirl69,6.0,0,6.0,6.0,25241032,February 2009,353.0,"February 27, 2009",False,824.0,1
10047,5.0,4.0,3.0,5.0,4.0,4.0,“Hotel Palomar gets everything right”,nicely designed hotel quiet clean decent sized...,Mattonholiday,6.0,0,6.0,6.0,14683661,March 2008,353.0,"March 31, 2008",False,64.0,1
10048,3.0,4.0,5.0,4.0,4.0,4.0,“Friendly staff-noisy doors everywhere”,pros\n-friendly staff particularly doorman val...,larklaw,43.0,2,54.0,43.0,11093658,November 2007,353.0,"November 27, 2007",False,470.0,0
